In [3]:
# default_exp algo.rs.match.matrix

%reload_ext autoreload
%autoreload 2

# 介绍
封装和rs相关的矩阵构建和分解的方法

# 矩阵构建

## 共现矩阵
以item_item_matrix的构建为例

一般共现矩阵比较稀疏

这里需要定义 什么是共现
* 是只要在一个list中就算是共现、
* 还是在list的一个window中算共现、
* 还是别的(如必须在item1出现之前的item才算与其共现)

还需要定义共现的权重
* 共现一次 +1
* 还是别的 方式，如与共现的距离相关，离的越远权重越小

In [3]:
#export
import scipy.sparse as sp
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

## build_co_occurance_matrix

In [24]:
#export

def build_co_occurance_matrix(items_list, 
                              window=9999,
                              penalty1=False,
                              penalty2=False,
                              penalty3=1, 
                              penalty4=False, 
                              norm=False,
                              save_dir=None):
    """
    矩阵index: next_items，columns: items
    共现矩阵的构建需要考虑很多因素。可以有多种构造方法: 
        如对于一个有序序列 [item1, item2, item4] item2肯定算作item1的共现，但是item1是否算作item2的共现呢？这里有两种处理方法:
            1 做惩罚。因为item1出现在item2之前，那么如果现在用户点击了item2，再点击item1的概率相对较低，所以要惩罚，如乘以一个小于1的惩罚系数
            2 干脆不算做共现。这个在item有严格的顺序的情况下是合理的，相当于1的特殊情况，即惩罚系数是0
    :items_list:
        [
            [item1, item2], 
            [item1, item2, item4], 
            ...
        ]
    : window: int
        只有在window内才算共现
    : penalty1:
        对距离惩罚，距离越远，相关性越小
    : penalty2:
        对list长度惩罚，长度越长，对共现的价值越小
    : penalty3: float
        对seq方向惩罚，方向为正 不惩罚，否则惩罚
        1表示不惩罚
    : penalty4:
        对item出现次数做惩罚，出现越多，对共现的价值越小
    : norm: 暂时没有实现。。。
        是否对向量做归一化。如果不做归一化，矩阵元素xij表示itemi和itemj共现的次数；列归一化后，表示itemj后出现itemi的概率
        0: 不做归一化
        1: 列归一化
        2: 行归一化
        
    :return:
        
    
    """
    from multiprocessing.dummy import Pool, Lock
    from collections import Counter
    
    pool = Pool()
    mutex = Lock()
    item_list_flat = [ii for i in items_list for ii in i]
    item_num_dict = dict(Counter(item_list_flat))  # 每个item出现次数的字典
    
    items = pd.Series(list(item_num_dict.keys()))
    item2id = pd.Series(items.index, items)
    
    n_items = items.shape[0]
    print(f'n_items: {n_items}')
    train_data_matrix = sp.lil_matrix((n_items, n_items), dtype=np.float)
    def t(items_):
#     for items_ in tqdm(items_list):
        for i, item in enumerate(items_):
            for j, related_item in enumerate(items_):
                distance = np.abs(i-j)
                if (item != related_item) and (distance<window):
                    vt = 1 
                    if penalty1:
#                         print('对距离惩罚，距离越远，相关性越小...')
                        vt /= np.sqrt(np.log2(distance+1))
                    if penalty2:
#                         print('对list长度惩罚，长度越长，对共现的价值越小...')
                        vt /= np.log10(len(items_)+9)
                    if i < j:
                        vt *= penalty3
                    mutex.acquire()
                    train_data_matrix[item2id.loc[item], item2id.loc[related_item]] += vt
                    mutex.release()
    pool.map(t, items_list)
    if penalty4:
        print('对item出现次数做惩罚...')
        def t(r):
#         for r in tqdm(range(train_data_matrix.shape[0])):
            for c in train_data_matrix.rows[r]:
                mutex.acquire()
                train_data_matrix[r,c] /= (np.log(item_num_dict[items[r]]+1)*np.log(item_num_dict[items[c]]+1))
                mutex.release()
        pool.map(t, range(train_data_matrix.shape[0]))
    if save_dir:
        if not os.path.exists(save_dir):
            print(f'create matrix dir {save_dir}')
            os.mkdir(save_dir)
        items.to_pickle(os.path.join(save_dir, f'id2item_series_{penalty1}_{penalty2}_{penalty3}.pkl'))
        item2id.to_pickle(os.path.join(save_dir, f'item2id_series_{penalty1}_{penalty2}_{penalty3}.pkl'))
        sp.save_npz(os.path.join(save_dir, f'item_item_matrix_{penalty1}_{penalty2}_{penalty3}.npz'), train_data_matrix.tocsc())
        print(f'save matrix to {save_dir}, finished')
    return train_data_matrix, items, item2id

## load_co_occurance_matrix

In [35]:
#export
def load_co_occurance_matrix(save_dir,penalty1,penalty2,penalty3):
    id2item = pd.read_pickle(os.path.join(save_dir, f'id2item_series_{penalty1}_{penalty2}_{penalty3}.pkl'))
    item2id = pd.read_pickle(os.path.join(save_dir, f'item2id_series_{penalty1}_{penalty2}_{penalty3}.pkl'))
    co_occurance_matrix = sp.load_npz(os.path.join(save_dir, f'item_item_matrix_{penalty1}_{penalty2}_{penalty3}.npz'))
    return co_occurance_matrix, id2item, item2id

## user_item_matrix

In [4]:
#export
def build_user_item_matrix(df, user_col, item_col):
    """
    使用pd.crosstab(df[user_col], df[item_col])可以直接达到目的，但是当items很大时，会报异常:
    ValueError: Unstacked DataFrame is too big, causing int32 overflow
    """
    
    n_users = df[user_col].nunique()
    n_items = df[item_col].nunique()
    id2user = df[user_col].drop_duplicates().reset_index(drop=True)
    user2id = pd.Series(id2user.index, id2user)
    id2item = df[item_col].drop_duplicates().reset_index(drop=True)
    item2id = pd.Series(id2item.index, id2item)
    print(f'n_users: {n_users}, n_items: {n_items}')
    train_data_matrix = sp.lil_matrix((n_users, n_items))
    for line in df[[user_col, item_col]].itertuples():
        train_data_matrix[user2id[line[1]], item2id[line[2]]] += 1
    train_data_matrix = train_data_matrix.tocsc()
    train_data_matrix.data = np.log(train_data_matrix.data + 1)
    return train_data_matrix, id2user, user2id, id2item, item2id

## 余弦相似矩阵

### build_item_item_cosine_matrix

In [30]:
#export
def build_item_item_cosine_matrix(matr, Y=None):
    """
    由于item一般数据很多(数10w)，需要很大的内存
    """
    from sklearn.metrics.pairwise import cosine_similarity
    return cosine_similarity(matr, Y)

In [33]:
matr = np.arange(8).reshape(2, 4)
matr

array([[0, 1, 2, 3],
       [4, 5, 6, 7]])

In [34]:
item_item_cosine_matrix(matr)

array([[1.       , 0.9047619],
       [0.9047619, 1.       ]])

# 矩阵分解FM
matrix factorization 

矩阵分解 (decomposition, factorization)是将矩阵拆解为数个矩阵的乘积。

常见的矩阵分解方法有三种：
* 1)三角分解法 (Triangular Factorization)，
* 2)QR 分解法 (QR Factorization)，
* 3)奇异值分解法 (Singular Value Decomposition)。


## 利用nn做矩阵分解
矩阵分解的一个基本要求就是分解以后的矩阵的乘积和原矩阵的相应位置的元素尽量相近！ 这个本身就能构造loss function。


如
$$M_{ab}=U_{ac}*V_{bc}^T$$

loss function为:
$$l = u_i.dot(v_j)-M_{ij}$$
如M为user_item矩阵，矩阵的每一个元素为user对该item的评分，那么分解后的U可以看做是用户矩阵，每一个行向量表示相应的用户向量；V可以看做是item矩阵，每一个行向量表示一个item。为了保证二者可以做内积，二者必须有相同的维度c。

### Computational Graph
#### 构造nn最简单的方式
可以看到，没有层的概念。。。
![cgmf](img/cgmf.png)

## 矩阵分解应用

### 推荐

### 语言模型
![](img/mf01.png)
![](img/mf11.png)

# LightFM
https://github.com/lyst/lightfm

https://making.lyst.com/lightfm/docs/home.html

Learning to Rank Sketchfab Models with LightFM 
https://www.ethanrosenthal.com/2016/11/07/implicit-mf-part-2/

LightFM是针对隐式和显式反馈的许多流行推荐算法的Python实现，包括BPR和WARP排名损失的有效实现。 它易于使用，快速（通过多线程模型估计）并产生高质量的结果。

它还使将item和用户元数据都合并到传统的矩阵分解算法中成为可能。 它将每个用户和item表示为其特征的潜在表示的总和，从而允许将recommendations推广到新item（通过item features）和新用户（通过用户features）。


## install

In [5]:
# !pip install lightfm -i https://pypi.tuna.tsinghua.edu.cn/simple
!pip freeze | grep lightfm

lightfm==1.15


## Quickstart
Fitting an implicit feedback model on the MovieLens 100k dataset is very easy:

In [6]:
from lightfm import LightFM
from lightfm.datasets import fetch_movielens
from lightfm.evaluation import precision_at_k

/Users/luoyonggui/anaconda3/lib/python3.7/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


In [7]:
# Load the MovieLens 100k dataset. Only five
# star ratings are treated as positive.
data = fetch_movielens(min_rating=5.0)

In [10]:
type(data)

dict

In [11]:
data

{'train': <943x1682 sparse matrix of type '<class 'numpy.int32'>'
 	with 19048 stored elements in COOrdinate format>,
 'test': <943x1682 sparse matrix of type '<class 'numpy.int32'>'
 	with 2153 stored elements in COOrdinate format>,
 'item_features': <1682x1682 sparse matrix of type '<class 'numpy.float32'>'
 	with 1682 stored elements in Compressed Sparse Row format>,
 'item_feature_labels': array(['Toy Story (1995)', 'GoldenEye (1995)', 'Four Rooms (1995)', ...,
        'Sliding Doors (1998)', 'You So Crazy (1994)',
        'Scream of Stone (Schrei aus Stein) (1991)'], dtype=object),
 'item_labels': array(['Toy Story (1995)', 'GoldenEye (1995)', 'Four Rooms (1995)', ...,
        'Sliding Doors (1998)', 'You So Crazy (1994)',
        'Scream of Stone (Schrei aus Stein) (1991)'], dtype=object)}

In [19]:
data['train'].toarray().shape

(943, 1682)

In [20]:
data['train'].toarray()[0]

array([5., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [15]:
# Instantiate and train the model
model = LightFM(loss='warp')
model.fit(data['train'], epochs=30, num_threads=2)

In [31]:
model.get_item_representations()[1].shape

(1682, 10)

In [32]:
len(model.get_user_representations())

2

In [30]:
model.get_user_representations()[1].shape

(943, 10)

In [34]:
model.user_embedding_gradients.shape

(943, 10)

In [35]:
model.predict_rank(data['test'])

<943x1682 sparse matrix of type '<class 'numpy.float32'>'
	with 2153 stored elements in Compressed Sparse Row format>

In [36]:
model.predict_rank(data['test']).shape

(943, 1682)

In [37]:
model.predict_rank(data['test']).toarray()[0]

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [ ]:
model.predict()

In [16]:
# Evaluate the trained model
test_precision = precision_at_k(model, data['test'], k=5).mean()

In [17]:
test_precision

0.051254958

## api

### LightFM

    LightFM(
        no_components=10,
        k=5,
        n=10,
        learning_schedule='adagrad',
        loss='logistic',
        learning_rate=0.05,
        rho=0.95,
        epsilon=1e-06,
        item_alpha=0.0,
        user_alpha=0.0,
        max_sampled=10,
        random_state=None,
    )

A hybrid(混合的) latent representation recommender model.

The model learns embeddings (latent representations in a high-dimensional space) for users and items in a way that encodes user preferences over items.

When multiplied together, these representations produce scores for every item for a given user; items scored highly are more likely to be interesting to the user.

The user and item representations are expressed in terms of representations of their features: an embedding is estimated for every feature, and these features are then summed together to arrive at representations for users and items. 

For example, if the movie 'Wizard of Oz' is described by the following features: 'musical fantasy', 'Judy Garland', and 'Wizard of Oz', then its embedding will be given by taking the features' embeddings and adding them together. The same applies to user features.

The embeddings are learned through `stochastic gradient descent <http://cs231n.github.io/optimization-1/>`_ methods.

Four loss functions are available:

- logistic: useful when both positive (1) and negative (-1) interactions
  are present.
- BPR: Bayesian Personalised Ranking [1]_ pairwise loss. Maximises the prediction difference between a positive example and a randomly chosen negative example. Useful when only positive interactions are present and optimising ROC AUC is desired.
- WARP: Weighted Approximate-Rank Pairwise [2]_ loss. Maximises
  the rank of positive examples by repeatedly sampling negative
  examples until rank violating one is found. Useful when only
  positive interactions are present and optimising the top of
  the recommendation list (precision@k) is desired.
- k-OS WARP: k-th order statistic loss [3]_. A modification of WARP that
  uses the k-th positive example for any given user as a basis for pairwise
  updates.

Two learning rate schedules are available:

- adagrad: [4]_
- adadelta: [5]_

#### Parameters

    no_components: int, optional
        the dimensionality of the feature latent embeddings.
    k: int, optional
        for k-OS training, the k-th positive example will be selected from the
        n positive examples sampled for every user.
    n: int, optional
        for k-OS training, maximum number of positives sampled for each update.
    learning_schedule: string, optional
        one of ('adagrad', 'adadelta').
    loss: string, optional
        one of  ('logistic', 'bpr', 'warp', 'warp-kos'): the loss function.
    learning_rate: float, optional
        initial learning rate for the adagrad learning schedule.
    rho: float, optional
        moving average coefficient for the adadelta learning schedule.
    epsilon: float, optional
        conditioning parameter for the adadelta learning schedule.
    item_alpha: float, optional
        L2 penalty on item features. Tip: setting this number too high can slow
        down training. One good way to check is if the final weights in the
        embeddings turned out to be mostly zero. The same idea applies to
        the user_alpha parameter.
    user_alpha: float, optional
        L2 penalty on user features.
    max_sampled: int, optional
        maximum number of negative samples used during WARP fitting.
        It requires a lot of sampling to find negative triplets for users that
        are already well represented by the model; this can lead to very long
        training times and overfitting. Setting this to a higher number will
        generally lead to longer training times, but may in some cases improve
        accuracy.
    random_state: int seed, RandomState instance, or None
        The seed of the pseudo random number generator to use when shuffling
        the data and initializing the parameters.

#### Attributes

    item_embeddings: np.float32 array of shape [n_item_features, n_components]
         Contains the estimated latent vectors for item features. The [i, j]-th
         entry gives the value of the j-th component for the i-th item feature.
         In the simplest case where the item feature matrix is an identity
         matrix, the i-th row will represent the i-th item latent vector.
    user_embeddings: np.float32 array of shape [n_user_features, n_components]
         Contains the estimated latent vectors for user features. The [i, j]-th
         entry gives the value of the j-th component for the i-th user feature.
         In the simplest case where the user feature matrix is an identity
         matrix, the i-th row will represent the i-th user latent vector.
    item_biases: np.float32 array of shape [n_item_features,]
         Contains the biases for item_features.
    user_biases: np.float32 array of shape [n_user_features,]
         Contains the biases for user_features.

#### Notes

Users' and items' latent representations are expressed in terms of their
features' representations. If no feature matrices are provided to the
:func:`lightfm.LightFM.fit` or :func:`lightfm.LightFM.predict` methods, they are
implicitly assumed to be identity matrices: that is, each user and item
are characterised by one feature that is unique to that user (or item).
In this case, LightFM reduces to a traditional collaborative filtering
matrix factorization method.

When a feature matrix is provided, it should be of shape
``(num_<users/items> x num_features)``. An embedding will then be estimated
for every feature: that is, there will be ``num_features`` embeddings.
To obtain the representation for user i, the model will look up the i-th
row of the feature matrix to find the features with non-zero weights in
that row; the embeddings for these features will then be added together
to arrive at the user representation. For example, if user 10 has weight 1
in the 5th column of the user feature matrix, and weight 3 in the 20th
column, that user's representation will be found by adding together
the embedding for the 5th and the 20th features (multiplying the latter
by 3). The same goes for items.

Note: when supplying feature matrices, an implicit identity feature
matrix will no longer be used. This may result in a less expressive model:
because no per-user features are estiamated, the model may underfit. To
combat this, include per-user (per-item) features (that is, an identity
matrix) as part of the feature matrix you supply.

# nb_export

In [2]:
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted 00_template.ipynb.
Converted algo_dl_keras.ipynb.
Converted algo_ml_shallow_tree_catboost.ipynb.
Converted algo_rs_associated_rules.ipynb.
Converted algo_rs_match_deepmatch.ipynb.
Converted algo_rs_matrix.ipynb.
Converted algo_rs_search_vector_faiss.ipynb.
Converted algo_seq_embeding.ipynb.
Converted algo_seq_tfidf.ipynb.
Converted datastructure_time.ipynb.
Converted engineering_concurrency.ipynb.
Converted engineering_nbdev.ipynb.
Converted engineering_panel.ipynb.
Converted engineering_snorkel.ipynb.
Converted index.ipynb.
Converted utils_json.ipynb.
Converted utils_pickle.ipynb.


In [7]:
!nbdev_build_docs

No notebooks were modified
converting /Users/luoyonggui/PycharmProjects/nbdevlib/index.ipynb to README.md
